# This notebook will be mainly used for the capstone project.

In [17]:
import pandas as pd
import numpy as np
import warnings
print("Hello Capstone Project Course")

Hello Capstone Project Course


In [18]:
df = pd.read_csv("Data-Collisions.csv")
warnings.filterwarnings('ignore')

# Introduction: Business Problem and Background
Seattle being one of the fastest-growing major United States cities has recently been experiencing an increase in the number of road accidents. As such, this had caught an attention in key stakeholders within the Seattle Department of Transport (SDOT). Noting the City’s long-term vision of making Seattle one of the most liveable cities in the world, the SDOT want detailed insights on the car accident severity (fatality) considering key factors that might have contributed to the accident.  The findings will be very useful as they will enable stakeholders to deduce data-driven strategies to prevent future road accidents and thus, consequently saving lives.

# Description of the Data
To solve this problem, the readily available shared “data-collision” dataset from SDOT will be used. The dataset has 38 attributes (columns), with the first column being the target column labelled “SEVERITYCODE”. The remaining 37 attributes are feature candidates to choose from when building the model. In total, the dataset contains 194 673 observations which are sufficient to train and test the machine learning model. The 37 attributes comprise categorial and numerical data. Key consideration will be taken when selecting feature columns, for instance, columns with very high numbers of missing values or are duplicates will be dropped e.g., SPEEDING (9333 non-null entries), EXCEPTRSNDESC (5638 non-null entries), PEDROWNOTGRNT (4667 non-null entries) and a duplicate column of SEVERITYCODE. On the other hand, feature columns including but not limited to weather condition (WEATHER), light conditions (LIGHTCOND), road condition (ROADCOND) junction type (JUNCTIONTYPE), number of people involved (PERSONCOUNT), number of vehicles involved (VEHCOUNT), and collision type (COLLISIONTYPE) will be viable candidates to consider when building the model. It is also worth noting that these columns have non-null values > 189 000. Finally, it is worth noting that the target label is very unbalanced ~ 7:3, hence the dataset needs to be balanced to avoid the effects of bias in the model.

In [15]:
df['SEVERITYCODE'].value_counts(normalize = True)

1    0.701099
2    0.298901
Name: SEVERITYCODE, dtype: float64